In [1]:
# import the necessary packages
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
from tracker import EuclideanDistTracker
from itertools import combinations
from non_max_suppresion import non_max_suppression_fast
from ipynb.fs.full.detect_people import yolo
from ipynb.fs.full.detect_mask import detect_and_predict_mask
from scipy.spatial import distance as dist
import social_distancing_config as config 


tracker = EuclideanDistTracker()
centroid_dict = dict()

net = cv2.dnn.readNet('yolo_tiny/yolov3-tiny.weights', 'yolo_tiny/yolov3-tiny.cfg')
#classes = ['person']
classes = []
with open("coco.names", "r") as f:
    classes = f.read().splitlines()
font = cv2.FONT_HERSHEY_PLAIN

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = r"face_detector/deploy.prototxt"
weightsPath = r"face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model('mask_detector.model')

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream('http://192.168.29.49:8080/video').start()
time.sleep(2.0)
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    ##Call YOLO (detect_people function)##
    
    results = yolo(frame, net, classes, font)
    
    # initialize the set of indexes that violate the minimum social distance
    violate = set()

    # ensure there are *at least* two people detections (required in
    # order to compute our pairwise distance maps)
    if len(results) >= 2:
        # extract all centroids from the results and compute the
        # Euclidean distances between all pairs of the centroids
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")

        # loop over the upper triangular of the distance matrix
        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                # check to see if the distance between any two
                # centroid pairs is less than the configured number
                # of pixels
                if D[i, j] < config.MIN_DISTANCE:
                    # update our violation set with the indexes of
                    # the centroid pairs
                    violate.add(i)
                    violate.add(j)

    # loop over the results
    for (i, (prob, bbox, centroid)) in enumerate(results):
        # extract the bounding box and centroid coordinates, then
        # initialize the color of the annotation
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid
        color = (0, 255, 0)

        # if the index pair exists within the violation set, then
        # update the color
        if i in violate:
            color = (0, 0, 255)
            text = str('Maintain Social Distancing!')
            cv2.putText(frame, text, (50, 30),
            font, 0.9, color, 2)

        # draw (1) a bounding box around the person and (2) the
        # centroid coordinates of the person,
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        #cv2.circle(frame, (cX, cY), 5, color, 1)

    
    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)  ##from detect_mask fuction

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            font, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # esc key
    if key == 27:
        break
`
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


[INFO] loading face detector model...
[INFO] loading face mask detector model...
[INFO] starting video stream...
